In [ ]:
import io, os, sys, types
import nbformat
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [ ]:
class NotebookLoader(object):
    """Module Loader for IPython Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing notebook from %s" % path)
                                       
        # load the notebook object
        nb = nbformat.read(path, as_version=4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [ ]:
class NotebookFinder(object):
    """Module finder that locates IPython Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [ ]:
sys.meta_path.append(NotebookFinder())

In [ ]:
from rating_prediction_lda import starsGroup,all_text_train, all_text_test,topic_dist_train_all_stars,topic_dist_test_all_stars

In [ ]:
# Extracting features using term frequency
tfidfvectorizer = TfidfVectorizer()

tfidfTrain = tfidfvectorizer.fit_transform(all_text_train)
tfidfTest = tfidfvectorizer.transform(all_text_test)

tfidfLabelTrain = topic_dist_train_all_stars['Star']
tfidfLabelTest = topic_dist_train_all_stars['Star']

In [ ]:
classifiers = [MultinomialNB(), LogisticRegression()]
classifiers_names = ['Multinomial Naive Bayes', 'Logistic Regression']

In [ ]:
TFIDF_Pred_Results = {}
for (i, clf_) in enumerate(classifiers):
    clf = clf_.fit(tfidfTrain, tfidfLabelTrain)
    prediction = clf.predict(tfidfTest)
    
    precision = metrics.precision_score(tfidfLabelTrain, prediction)
    recall = metrics.recall_score(tfidfLabelTest, prediction)
    f1 = metrics.f1_score(tfidfLabelTest, prediction)
    accuracy = accuracy_score(tfidfLabelTest, prediction)
    report = classification_report(tfidfLabelTest, prediction)
    matrix = metrics.confusion_matrix(tfidfLabelTest, prediction, labels=starsGroup.groups.keys())
    
    data = {'precision':precision,
            'recall':recall,
            'f1_score':f1,
            'accuracy':accuracy,
            'clf_report':report,
            'clf_matrix':matrix,
            'y_predicted':preds}
    
    TFIDF_Pred_Results[classifiers_names[i]] = data

In [ ]:
cols = ['precision', 'recall', 'f1_score', 'accuracy']
pd.DataFrame(TFIDF_Pred_Results).T[cols].T

In [ ]:
for model, val in TFIDF_Pred_Results.iteritems():
    print '-------'+'-'*len(model)
    print 'MODEL:', model
    print '-------'+'-'*len(model)
    print 'The precision for this classifier is ' + str(val['precision'])
    print 'The recall for this classifier is    ' + str(val['recall'])
    print 'The f1 for this classifier is        ' + str(val['f1_score'])
    print 'The accuracy for this classifier is  ' + str(val['accuracy'])
    print 'The confusion matrix for this classifier is  \n' + str(val['clf_matrix'])
    print '\nHere is the classification report:'
    print val['clf_report']